---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [126]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    

    towns = pd.read_csv('university_towns.txt',delimiter='\t',header=None)
    towns.columns = ['lst']     #Rename column
    lst = (towns.lst)               #make lst a tuple

    def separate_states_regions(lst, towns):
        for line in lst:
            if '[ed' in line:
                towns['State'] = towns['lst'].apply(lambda x: x.split('[')[0].strip() if x.count('[edit') > 0 else np.NaN).fillna(method="ffill")
            else:
                if '(' in line:
                    towns['RegionName'] = towns['lst'].apply(lambda x: np.NaN if x.count('[edit') > 0 else x.split('(')[0].strip())
                else:    
                    towns['RegionName'] = line.strip('\n')
        return towns[['State', 'RegionName']]

    towns = separate_states_regions(lst, towns)
    towns.dropna(inplace = True)
    
    #states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    #now clean up data, then map cities with states. Use previous assignment for guidin on mapping 
    return towns

get_list_of_university_towns()


State                    RegionName
1          Alabama                        Auburn
2          Alabama                      Florence
3          Alabama                  Jacksonville
4          Alabama                    Livingston
5          Alabama                    Montevallo
6          Alabama                          Troy
7          Alabama                    Tuscaloosa
8          Alabama                      Tuskegee
10          Alaska                     Fairbanks
12         Arizona                     Flagstaff
13         Arizona                         Tempe
14         Arizona                        Tucson
16        Arkansas                   Arkadelphia
17        Arkansas                        Conway
18        Arkansas                  Fayetteville
19        Arkansas                     Jonesboro
20        Arkansas                      Magnolia
21        Arkansas                    Monticello
22        Arkansas                  Russellville
23        Arkansas                        Searcy
25      California                        Angwin
26      California                        Arcata
27      California                      Berkeley
28      California                         Chico
29      California                     Claremont
30      California                        Cotati
31      California                         Davis
32      California                        Irvine
33      California                    Isla Vista
34      California  University Park, Los Angeles
..             ...                           ...
533       Virginia                          Wise
534       Virginia                    Chesapeake
536     Washington                    Bellingham
537     Washington                        Cheney
538     Washington                    Ellensburg
539     Washington                       Pullman
540     Washington  University District, Seattle
542  West Virginia                        Athens
543  West Virginia                    Buckhannon
544  West Virginia                      Fairmont
545  West Virginia                     Glenville
546  West Virginia                    Huntington
547  West Virginia                    Montgomery
548  West Virginia                    Morgantown
549  West Virginia                 Shepherdstown
550  West Virginia                  West Liberty
552      Wisconsin                      Appleton
553      Wisconsin                    Eau Claire
554      Wisconsin                     Green Bay
555      Wisconsin                     La Crosse
556      Wisconsin                       Madison
557      Wisconsin                     Menomonie
558      Wisconsin                     Milwaukee
559      Wisconsin                       Oshkosh
560      Wisconsin                   Platteville
561      Wisconsin                   River Falls
562      Wisconsin                 Stevens Point
563      Wisconsin                      Waukesha
564      Wisconsin                    Whitewater
566        Wyoming                       Laramie

[517 rows x 2 columns]

In [143]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    yr_qt = pd.read_excel( "gdplev.xls", skiprows =219)
    yr_qt.drop( yr_qt.columns[ [ 0,1,2,3, 5, 7 ] ], axis =1, inplace = True )    #Drop specific columns
    yr_qt.columns = ['Time', 'Quart GDP [$B 2009]']
    quart = yr_qt[ 'Quart GDP [$B 2009]']  #Copy gdp column

    for j in range(1,len(quart)-1):    #length of for loop 
        if quart[j +1] < quart[j] < quart[j -1]: #Two consecutive quarters
            start = yr_qt[ 'Time'][j] 
            break
            
    
    return start
get_recession_start()

'2008q3'

In [38]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    yr_qt = pd.read_excel( "gdplev.xls", skiprows =219)
    yr_qt.drop( yr_qt.columns[ [ 0,1,2,3, 5, 7 ] ], axis =1, inplace = True )    #Drop specific columns
    yr_qt.columns = ['Time', 'Quart GDP [$B 2009]']    #need to redefine starting point from 2008
    start = yr_qt[yr_qt.Time == "2008q2"].index[0]
    quart = yr_qt.loc[start:]    #starting from 2008q2


    for j in range(1,len(quart)-1):    #length of for loop 
        if quart[ 'Quart GDP [$B 2009]'].iloc[j+1] > quart[ 'Quart GDP [$B 2009]'].iloc[j] > quart[ 'Quart GDP [$B 2009]'].iloc[j -1]:
            end = quart[ 'Time'].iloc[j+1] 
            break        
    
    return end

get_recession_end()   

'2009q4'

In [128]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    yr_qt = pd.read_excel( "gdplev.xls", skiprows =219)
    yr_qt.drop( yr_qt.columns[ [ 0,1,2,3, 5, 7 ] ], axis =1, inplace = True )    #Drop specific columns
    yr_qt.columns = ['Time', 'Quart GDP [$B 2009]']
    quart = yr_qt[ 'Quart GDP [$B 2009]']  #Copy gdp column
    #yr_qt.index.get_loc('2000q1')
    start = yr_qt[yr_qt.Time == "2008q2"].index[0]
    end = yr_qt[yr_qt.Time == "2009q4"].index[0]
    bot_ind = np.argmin(yr_qt[ 'Quart GDP [$B 2009]'].loc[start:end])
    bottom =  yr_qt.Time[bot_ind]

    
    return bottom

get_recession_bottom()

'2009q2'

In [140]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv',header=0)
    housing.drop( housing.columns[ [ 0,3,4,5 ] ], axis =1, inplace = True )
    housing.set_index('State', inplace = True)
    
    housing.drop(housing.columns[1:46], axis=1, inplace = True)            #Keep columns from 2001 and on
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    st = pd.DataFrame([states]).T  #Convert Sates into dataframe 
    df = pd.merge(st, housing, how='inner', left_index=True, right_index=True)
    df.rename(columns={0: 'State'} , inplace = True)
    df.set_index(['State', 'RegionName'], inplace=True)  #mutli index 

    df.columns = pd.to_datetime(df.columns)  #Convet to datetime for period 
    df = df.groupby(df.columns.to_period('Q'), axis=1).mean()  #Groups and averages columns
    
    
    df.columns = df.columns.to_series().astype(str).str.lower()    #format the column headings to lowercase strings
    
     

    return df
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State         RegionName                                                      
Alaska        Anchorage         174633.333333  175266.666667  179566.666667   
              Fairbanks         163200.000000  165033.333333  169300.000000   
              Palmer            168766.666667  170566.666667  174233.333333   
              North Pole        157466.666667  159100.000000  162133.333333   
              Juneau            192466.666667  194300.000000  195166.666667   
              Kenai                       NaN            NaN            NaN   
              Soldotna                    NaN            NaN            NaN   
              Kodiak                      NaN            NaN            NaN   
              Tanaina           160500.000000  163500.000000  166366.666667   
              Ketchikan                   NaN            NaN            NaN   
              Lakes             172733.333333  174133.333333  177266.666667   
              Homer                       NaN            NaN            NaN   
              Seward                      NaN            NaN            NaN   
              Anchor Point                NaN            NaN            NaN   
Alabama       Mobile             78233.333333   79366.666667   80433.333333   
              Birmingham         54033.333333   54400.000000   54966.666667   
              Huntsville                  NaN            NaN            NaN   
              Tuscaloosa        104500.000000  106033.333333  108500.000000   
              Decatur                     NaN            NaN            NaN   
              Hoover            184966.666667  186366.666667  187200.000000   
              Madison                     NaN            NaN            NaN   
              Northport         125866.666667  126366.666667  126633.333333   
              Wetumpka                    NaN            NaN            NaN   
              Fairhope          161400.000000  161933.333333  161633.333333   
              Albertville        73966.666667   72600.000000   72833.333333   
              Alabaster         122133.333333  123066.666667  123166.666667   
              Prichard                    NaN            NaN            NaN   
              Theodore           76900.000000   78300.000000   78066.666667   
              Daphne            123166.666667  120233.333333  122300.000000   
              Homewood          158333.333333  159666.666667  161100.000000   
...                                       ...            ...            ...   
Wisconsin     Holland           151033.333333  150500.000000  153233.333333   
West Virginia Parkersburg        54900.000000   55833.333333   56633.333333   
              Charleston         59533.333333   59366.666667   59366.666667   
              Elkview                     NaN            NaN            NaN   
              South Charleston            NaN            NaN            NaN   
              Vienna             72266.666667   74533.333333   75333.333333   
              Cross Lanes        78133.333333   78066.666667   79233.333333   
              Saint Albans       63833.333333   64700.000000   65200.000000   
              Dunbar                      NaN            NaN            NaN   
              Nitro                       NaN            NaN            NaN   
              Williamstown       82933.333333   85200.000000   86466.666667   
              Clendenin          39966.666667   38200.000000   40500.000000   
              Sissonville        57200.000000   58200.000000   61200.000000   
              Washington         83900.000000   86033.333333   88333.333333   
              Davisville                  NaN            NaN            NaN   
              Mineral Wells      75533.333333   77233.333333   81966.666667   
              Upper Falls        53033.333333   52333.333333   52100.000000   
              Chesapeake                  NaN            NaN            NaN   
          

In [182]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()     #Call previous answer 
    df['price_ratio'] = df['2008q2'] / df['2009q2']  #define ratio
    df.reset_index(inplace = True)  #reset index for merger and seperation of uni towns 
    
    uni_towns = get_list_of_university_towns() #Call Uni towns
    Uni = pd.merge(uni_towns, df, how='inner', on=['State', 'RegionName']) # Create the interesection of University towns and df
    
    Non_uni = df[~df.RegionName.isin(uni_towns['RegionName'])] # Subtract out university towns, makes data frame
    
    
    uni_ratio = Uni["price_ratio"].dropna() #price ratio lists with no NaN
    Non_uni_ratio = Non_uni["price_ratio"].dropna()  
    
    def different():  #     """this function will return True if the uni and non-uni towns have a different change p<0.01"""
        t, p = ttest_ind(uni_ratio, Non_uni_ratio, equal_var = False)
        if p < 0.01:
            return True
        else:
            return False
        
    def p():   # Returns p value 
        t, p = ttest_ind(uni_ratio, Non_uni_ratio, equal_var = False)
        return p
    
    def better():
        if uni_ratio.mean() < Non_uni_ratio.mean():
            return "university town"
        else: 
            return "non-university town"
    
    return (different(),p(),better())
run_ttest()

(True, 0.00025784241689405215, 'university town')